In [55]:
# == importando bibliotecas == 

import pandas as pd
import numpy as np
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [202]:
artigos = {'Conteudo': ["""Ao participar da abertura do 26° encontro do Foro de São Paulo, o presidente Luiz Inácio Lula da Silva criticou a “direita fascista”, pediu união da esquerda no Brasil e disse que tem orgulho de ser chamado de “comunista”. Na cerimônia, ocorrida nesta quinta-feira, 29, em Brasília, o petista se reuniu com os partidos de esquerda da América Latina. Em seu discurso, o petista admitiu que o “discurso do patriotismo e da familia” não estão alinhados com o progressismo. “Aqui no Brasil, nós enfrentamos o discurso do costume, o discurso da família, o discurso do patriotismo. Ou seja, aqui nós enfrentamos o discurso que a gente aprendeu a historicamente combater”, afirmou o presidente da República. “Eles nos acusam de comunistas, achando que nós ficamos ofendidos com isso. Nós não ficamos ofendidos. Ficaríamos ofendidos se nos chamassem de nazista, de neofascista, de terrorista. Mas de comunista, de socialista, nunca. Isso não nos ofende. Isso nos orgulha muitas vezes”, continuou o petista.

Fundado em 1990, por iniciativa do Partido dos Trabalhadores (PT) do Brasil e do Partido Comunista de Cuba, o Foro de São Paulo é um encontro dos partidos de esquerda da América Latina e tem como lema neste ano “Integração Regional para Avançar a Soberania Latino-americana e Caribenha”. Após a pandemia, o grupo retoma suas reuniões presenciais anuais. A última edição ocorreu em Caracas , capital da Venezuela, em 2019. Além de Lula, estiverem presentes a presidente do PT, Gleisi Hoffmann, a ministra da Ciência e Tecnologia e presidente do PCdoB, Luciana Santos, e o presidente da Embratur, Marcelo Freixo. Manifestantes contrários ao encontro protestaram na porta da sede do evento. Além disso, partidos de direita, como o PL, repudiaram a presença do que chamam de “ditadores”. No Twitter, a oposição levantou a hashtag #ForaForodeSP.""".replace("\n\n", " ")]}
artigos = pd.DataFrame(artigos)

In [203]:
artigos.Conteudo.iloc[0]

'Ao participar da abertura do 26° encontro do Foro de São Paulo, o presidente Luiz Inácio Lula da Silva criticou a “direita fascista”, pediu união da esquerda no Brasil e disse que tem orgulho de ser chamado de “comunista”. Na cerimônia, ocorrida nesta quinta-feira, 29, em Brasília, o petista se reuniu com os partidos de esquerda da América Latina. Em seu discurso, o petista admitiu que o “discurso do patriotismo e da familia” não estão alinhados com o progressismo. “Aqui no Brasil, nós enfrentamos o discurso do costume, o discurso da família, o discurso do patriotismo. Ou seja, aqui nós enfrentamos o discurso que a gente aprendeu a historicamente combater”, afirmou o presidente da República. “Eles nos acusam de comunistas, achando que nós ficamos ofendidos com isso. Nós não ficamos ofendidos. Ficaríamos ofendidos se nos chamassem de nazista, de neofascista, de terrorista. Mas de comunista, de socialista, nunca. Isso não nos ofende. Isso nos orgulha muitas vezes”, continuou o petista. 

In [204]:
# == ler a base de artigos == 

#artigos = pd.read_parquet('../../dataset/processed/artigos_de_partidos/artigos_partidos.parquet')

In [205]:
# == tornar os artigos em letras minúsculas == 

artigos.Conteudo = artigos.Conteudo.str.lower()

In [206]:
# == remover artigos contendo links == 

#artigos[artigos.Conteudo.str.contains('http')].Partido.value_counts()
#artigos = artigos[~artigos.Conteudo.str.contains('http')]

In [207]:
# == todas as sentenças após o penúltimo "." a fim de eliminar ruídos que não fazem parte do artigo == 

artigos.Conteudo = artigos.Conteudo.apply(
    lambda x : ''.join(x.split('.')[:-2])
)

In [208]:
# == mascarar nome de partido == 

partidos = ['novo', 'pcb', 'pdt', 'pl', 'pp', 'psol', 'pstu', 'mdb', 'pcdob', 'psb', 'pt', 'pv', 'rede', 'união brasil']
for partido in partidos:
    artigos.Conteudo = artigos.Conteudo.str.replace(f' {partido} ', ' ')

In [209]:
# == remover todas as palavras contendo caracteres especiais ==

artigos.Conteudo = artigos.Conteudo.apply(
    lambda x : ' '.join(word for word in x.split(' ') if word.isalnum())
)

In [210]:
# == normalização por unicode e encode ascii ==

artigos.Conteudo = artigos.Conteudo.str.lower()\
                                   .str.normalize('NFKD')\
                                   .str.encode('ascii', errors='ignore')\
                                   .str.decode('utf-8')

In [211]:
# == remover pontuações e números == 

artigos.Conteudo = artigos.Conteudo.str.replace('[^a-zA-Z]', ' ')

In [212]:
# == remover double space == 

for n in range(30):
    artigos.Conteudo = artigos.Conteudo.str.replace('  ', ' ')

In [213]:
# == analisar a contagem média de palavras por partido == 

#copy = artigos.copy()
#copy['Contagem'] = copy.Conteudo.str.count(' ')
#copy.groupby(
#    by=['Vies', 'Partido'],
#).agg(
#    ContagemPalavras = ('Contagem', 'mean'), 
#    ContagemArtigos = ('Contagem', 'count')
#)

In [214]:
# == salvar o df tratado == 

artigos.to_csv('../../dataset/processed/artigos_tratados/artigo-direita4-.csv')